# Ticket/Request Triage with Structured Outputs

Goal: turn messy ticket/email text into **validated JSON** that your team can route, prioritize, and act on.

What you’ll practice:
- Defining a schema with Pydantic
- Using `client.responses.parse(...)` for structured outputs
- Quality gates: confidence thresholds + human review routing
- Batch processing pattern


## 1. Setup and Installation

**Estimated time:** ~60–90 minutes (with exercises)

### Install
If needed, install dependencies:
```bash
pip install -U openai pydantic pandas numpy scikit-learn
```

### Environment
Set your API key:
```bash
export OPENAI_API_KEY="..."
```

> **Note:** All example data in this notebook is synthetic (safe to share in training).

In [ ]:
import os

assert os.getenv('OPENAI_API_KEY'), "Set OPENAI_API_KEY in your environment"

## 2. Imports + API client

In [ ]:
from openai import OpenAI

client = OpenAI()  # uses OPENAI_API_KEY from env

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, List, Optional
import json
import pandas as pd
import re


## 3. Define a triage schema (Pydantic)

We’ll extract a *small, dependable* set of fields that are immediately useful in real workflows.


In [ ]:
class TicketTriage(BaseModel):
    ticket_id: str = Field(..., description="Ticket identifier, e.g., INC0012345 or JIRA-123")
    short_summary: str = Field(..., description="One-sentence summary in plain English")
    category: Literal["Accounts & Access","Website/App","Network","Hardware","Data/Reporting","Patron Services","Other"]
    priority: Literal["P0","P1","P2","P3"] = Field(..., description="P0 highest urgency")
    impact: Literal["Single user","Small group","Organization-wide","Public-facing outage","Unknown"]
    suggested_assignment_group: Literal["Help Desk","Web Team","Infra/SRE","Data","Security","Patron Services","Unknown"]
    recommended_next_actions: List[str] = Field(..., description="Concrete next steps, 1–5 items")
    needs_human_review: bool = Field(..., description="True if low confidence, ambiguous, or risky")
    confidence: float = Field(..., ge=0, le=1, description="0–1 confidence in the triage")
    pii_present: bool = Field(..., description="True if text contains PII (emails, phone numbers, patron names, addresses)")
    redaction_suggestion: Optional[str] = Field(None, description="How to redact if PII present")


## 4. Sample tickets (synthetic NYPL-style)

These are intentionally messy—like real inboxes.


In [ ]:
tickets = [
  {
    "ticket_id": "INC0012048",
    "text": "Patron can't reset password for MyNYPL. Error: 'token expired' even after retry. Email: jane.doe@example.com. Started today around 9am."
  },
  {
    "ticket_id": "JIRA-882",
    "text": "Intermittent 502s on /digital-collections/search since yesterday. Spikes around noon. Looks like upstream timeout."
  },
  {
    "ticket_id": "INC0012099",
    "text": "Staff laptop: Wi-Fi drops every 5 minutes on 3rd floor. Other devices OK. Tried rebooting router (no change)."
  },
  {
    "ticket_id": "INC0012112",
    "text": "Monthly circulation report is missing Bronx locations. Might be a data pipeline issue after last week's deploy."
  }
]

pd.DataFrame(tickets)

## 5. Baseline: simple deterministic triage (for comparison)

A tiny rules baseline helps you sanity-check the model output and explains what’s happening “behind the scenes”.


In [ ]:
def baseline_priority(text: str) -> str:
    t = text.lower()
    if any(k in t for k in ["outage","down","502","500","can’t access","cannot access"]):
        return "P1"
    if any(k in t for k in ["password","reset","login","sign in"]):
        return "P2"
    if any(k in t for k in ["wifi","network","drops"]):
        return "P2"
    return "P3"

for t in tickets:
    print(t["ticket_id"], baseline_priority(t["text"]))

## 6. Single-call triage with Structured Outputs

We ask for the schema and let the SDK validate the output.


In [ ]:
SYSTEM = """You are an IT/service triage assistant for a public library.
Return ONLY structured data that matches the provided schema.
Be conservative: if ambiguous or low confidence, set needs_human_review=true and confidence<=0.6.
If PII is present, mark pii_present=true and suggest how to redact."""

def triage_one(ticket_id: str, text: str) -> TicketTriage:
    response = client.responses.parse(
        model="gpt-4o-2024-08-06",
        input=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": f"Ticket ID: {ticket_id}\n\n{text}"}
        ],
        text_format=TicketTriage,
    )
    return response.output_parsed

triage = triage_one(tickets[0]["ticket_id"], tickets[0]["text"])
triage

## 7. Quality gates: confidence + human review routing

In production, you typically **don’t** auto-route everything. You route what’s high-confidence and send the rest to humans.


In [ ]:
def route(triage: TicketTriage) -> str:
    if triage.needs_human_review or triage.confidence < 0.75:
        return "HUMAN_REVIEW_QUEUE"
    return triage.suggested_assignment_group

print("Route:", route(triage))

## 8. Batch triage (common workflow)

Pattern:
1) loop tickets
2) parse structured output
3) store results as a table
4) apply routing logic


In [ ]:
rows=[]
for t in tickets:
    out = triage_one(t["ticket_id"], t["text"])
    rows.append(out.model_dump() | {"route": route(out)})

df = pd.DataFrame(rows)
df

## 9. Mini-evaluation: quick checks

A few simple checks catch many issues early.
- Required fields populated
- Confidence within [0,1]
- Priority is one of P0–P3


In [ ]:
assert df["confidence"].between(0,1).all()
assert set(df["priority"]).issubset({"P0","P1","P2","P3"})
df[["ticket_id","priority","category","suggested_assignment_group","route","confidence"]]

## 10. Exercises

Do these in order. In the **solutions** notebook, the filled answers are included.


In [ ]:

# EXERCISE — SOLUTION
# Add a new category called 'Facilities' and update the schema + prompt so the model can use it (e.g., elevators, HVAC, doors). Then add 1 synthetic ticket and re-run batch triage.

from typing import Literal

class TicketTriageV2(TicketTriage):
    category: Literal[
        "Accounts & Access","Website/App","Network","Hardware",
        "Data/Reporting","Patron Services","Facilities","Other"
    ]

SYSTEM_V2 = SYSTEM + "\nYou may use the category Facilities for building-related issues (elevators, HVAC, doors, lighting)."

def triage_one_v2(ticket_id: str, text: str) -> TicketTriageV2:
    response = client.responses.parse(
        model="gpt-4o-2024-08-06",
        input=[
            {"role": "system", "content": SYSTEM_V2},
            {"role": "user", "content": f"Ticket ID: {ticket_id}\n\n{text}"}
        ],
        text_format=TicketTriageV2,
    )
    return response.output_parsed

# Add a Facilities ticket
tickets_plus = tickets + [{
    "ticket_id": "INC0013001",
    "text": "Elevator stuck between floors at Mid-Manhattan. Staff cannot move book carts. Needs urgent facilities support."
}]

rows=[]
for t in tickets_plus:
    out = triage_one_v2(t["ticket_id"], t["text"])
    rows.append(out.model_dump() | {"route": ("HUMAN_REVIEW_QUEUE" if (out.needs_human_review or out.confidence < 0.75) else out.suggested_assignment_group)})

import pandas as pd
pd.DataFrame(rows)[["ticket_id","category","priority","impact","suggested_assignment_group","route","confidence"]]


In [ ]:
# EXERCISE — SOLUTION
# Implement a post-processor that upgrades priority to P1 if the model says 'Public-facing outage' impact, regardless of predicted priority.

def enforce_priority(triage: TicketTriage) -> TicketTriage:
    d = triage.model_dump()
    if d.get("impact") == "Public-facing outage" and d.get("priority") not in ("P0","P1"):
        d["priority"] = "P1"
    return TicketTriage(**d)

fixed=[]
for t in tickets:
    out=triage_one(t["ticket_id"], t["text"])
    fixed.append(enforce_priority(out).model_dump())

pd.DataFrame(fixed)[["ticket_id","impact","priority"]]


In [ ]:
# EXERCISE — SOLUTION
# Add a tiny unit-test style check that flags any output where `pii_present=False` but the text contains an email address.

EMAIL_RE = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")

violations=[]
for t in tickets:
    out=triage_one(t["ticket_id"], t["text"])
    has_email = bool(EMAIL_RE.search(t["text"]))
    if has_email and (out.pii_present is False):
        violations.append(t["ticket_id"])

print("Violations:", violations)
